#Extract
Authors: Gustavo FLEURY && Induraj RAMAMURTHY

Project: https://github.com/gustavofleury/Audit_Reports_NLP
#Take the raw['content'] in DF and transform Columns


In [0]:
#Libraries
import pandas as pd
import unicodedata as ud
import re

#Paths
DATASETS_FOLDER = '/content/drive/My Drive/Research/DataSets/'
FINAL_FILE_DF = DATASETS_FOLDER + 'DF_ReportsInColumns_ARG_v19.11.07.pkl'

In [0]:
#Take Files From GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import FILES
#List of Files - Combine AAG, AAC and ARG
# lstTypeReport=['AAC','AAG','ARG']
lstTypeReport=['ARG'] # Test
dfFile= pd.DataFrame()
for typeReport in lstTypeReport:
  fileName = DATASETS_FOLDER + "DF_Links_Reports_" + typeReport + ".pkl"
  if dfFile.empty:
    dfFile=pd.read_pickle(fileName)
  else:
    print('passed')
    dfFile=dfFile.append(pd.read_pickle(fileName), ignore_index=True)

#DF of Reports already Parsed
df_file_STR_Report = DATASETS_FOLDER + "DF_Reports_STR.pkl"
dfSTR=pd.read_pickle(df_file_STR_Report)

#DF of list of OrgSub (From Expenses DataBase)
dfOrgSub=pd.read_pickle( DATASETS_FOLDER + "OrgaoSubordinado_Unique.pkl")


In [0]:
#Some Info about Files
print('Files just STR    : ' + str(len(dfSTR)))
print('Files with Columns: ' + str(len(dfFile)))
print('Files with Columns: ' + str(len(dfFile)))



Files just STR    : 10879
Files with Columns: 1058
Files with Columns: 1058


In [0]:
# dfSTR.head(2)
dfFile.head(20)
# dfSTR['FileName'][ dfFile.Links.str.split('/').apply(lambda x: x[-1]) ].reset_index(drop=True)
# dfFile.Links.str.split('/').apply(lambda x: x[-1])
# dfSTR['FileName']=='13175.pdf'

In [0]:
#
def cleanReport(report):
  #Normalize, delete excess /N and spaces, Lowercase,
  report=ud.normalize('NFKD', report).encode('ASCII','ignore').decode('ASCII').strip().lower()
  report=report.replace('\n', '')
  report=re.sub(' +', ' ', report)

  #Repeated Elements to Delete (like page headers and footers notes)
  list2Delete=['dinheiro publico e da sua conta',
              'www.portaldatransparencia.gov.br',
              'presidencia da republica', 
              'controladoria-geral da uniao',
              'secretaria federal de controle interno'
              ]
  for text in list2Delete:
    report=report.replace(text,'')

  report=re.sub(' +', ' ', report)
  report=re.sub(' - ', ' ', report)
  report=re.sub('________________', '_', report)

  return report

#
def findReportType(fileNumber, dfLinks):  
  reportType =  dfLinks['Linha de Atuação'][ dfLinks['Links'].str.contains("/"+fileNumber+".pdf") ].reset_index(drop=True)
  
  return reportType[0]


In [0]:
#
def findHeader(report):
  #GERAL
  startText='(unidade\sauditada:\s|unidade\sauditada\s:\s|relatorio\sde\savaliacao)'
  # startText='(unidade\sauditada:\s|unidade\sauditada\s:\s)'
  endText='(escopo\sdo\strabalho|sumario\s)'
  # endText='(resultados\sdos\strabalhos|resultado\sdos\sexames)'

  #Relatorio de Fiscalizacao
  startTextRF='(relatorio)'
  endTextRF='(2\.\sresultados\sdos\sexames|ii\sresultado\sdos\sexames|ii\sresultados\sdos\sexames)'
  
  #Report ARG (example: 13175)
  startTextARG='(unidade\sauditada:\s|unidade\sauditada\s:\s)'
  endTextARG='(2\.\sresultados\sdos\strabalhos|2\.\sresultado\sdos\strabalhos)'

  if bool(re.search(startText + '(.*?)'+ endText,report)) :
    reportHEADER=re.search(startText + '(.*?)'+ endText,report)[0]
    # reportHEADER=re.findall(startText + '(.*?)'+ endText,report)[0][1]
  
  elif bool(re.search(startTextRF + '(.*?)'+ endTextRF, report)) :
    reportHEADER=re.search(startTextRF + '(.*?)'+ endTextRF, report)[0]
    # reportHEADER=re.findall(startTextRF + '(.*?)'+ endTextRF, report)[0][1]
  
  elif bool(re.search(startTextARG + '(.*?)'+ endTextARG, report)) :
    reportHEADER=re.search(startTextARG + '(.*?)'+ endTextARG, report)[0]
  
  else:
    reportHEADER="ERROR: Header not Found"
  return reportHEADER


In [0]:
#
def findReportNumber(reportHEADER):
  startText="(relatorio\sno:|relatorio\sno\s:|ordem\sde\sservico:|relatorio\sno|numero\sdo\srelatorio\s:)"
  if bool(re.search(startText+'\s(\d+)', reportHEADER)) :
    reportNum=re.findall(startText+'\s(\d+)', reportHEADER)[0][1]
  else:
    reportNum="ERROR"
  return reportNum

In [0]:
#
def findPeriod(reportHEADER, reportNum):
  month = "(janeiro|fevereiro|marco|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)"
  monthDict={'janeiro':'1','fevereiro':'2','marco':'3','abril':'4','maio':'5','junho':'6','julho':'7','agosto':'8','setembro':'9','outubro':'10','novembro':'11','dezembro':'12'}

  # no período de 01 de janeiro a 31 de dezembro de 2014
  if  bool(re.search('de\s(\d{2})\sde\s'+month+'\sa\s', reportHEADER)) and bool(re.search('a\s(\d{2})\sde\s'+month+'\sde\s(\d{4})', reportHEADER)) :
    dateBEGIN=re.findall('de\s(\d{2})\sde\s'+month+'\sa\s', reportHEADER)
    dateEND=re.findall('a\s(\d{2})\sde\s'+month+'\sde\s(\d{4})', reportHEADER)
    reportYear=dateEND[0][2]
    reportMesBegin=monthDict[dateBEGIN[0][1]]
    reportMesEnd=monthDict[dateEND[0][1]]
    reportBegin=reportYear+'/'+reportMesBegin
    reportEnd=reportYear+'/'+reportMesEnd

  # no exercício de 2014
  elif bool(re.search('no\sexercicio\sde\s(\d{4})', reportHEADER)):
    reportYear=re.findall('no\sexercicio\sde\s(\d{4})', reportHEADER)[0]
    reportBegin=reportYear+'/1'
    reportEnd=reportYear+'/12'

  # de 01/01/2014 a 31/12/2014
  elif bool(re.search('de\s(\d{2})/(\d{2})/(\d{4})\sa', reportHEADER)) and bool(re.search('a\s(\d{2})/(\d{2})/(\d{4})', reportHEADER)) :
    dateBEGIN=re.findall('de\s(\d{2})/(\d{2})/(\d{4})\sa', reportHEADER)
    dateEND=re.findall('a\s(\d{2})/(\d{2})/(\d{4})', reportHEADER)
    reportYearBEGIN=dateBEGIN[0][2]
    reportYearEND=dateEND[0][2]
    reportMesBegin=str(int(dateBEGIN[0][1]))
    reportMesEnd=str(int(dateEND[0][1]))
    reportBegin=reportYearBEGIN+'/'+reportMesBegin
    reportEnd=reportYearEND+'/'+reportMesEnd
  
  # Exercício 2018 (Relatório de Avaliação)
  elif "relatorio de avaliacao" in reportHEADER and bool(re.search('exercicio\s(\d{4})', reportHEADER)):
    reportYear=re.findall('exercicio\s(\d{4})', reportHEADER)
    reportBegin=str(reportYear[0]) + '/1'
    reportEnd=str(reportYear[0]) +'/12'

  # If not Found, take de last year from date of Report
  elif "ERR" not in reportNum:
    reportBegin= str(int(reportNum[0:4])-1) + '/1'
    reportEnd= str(int(reportNum[0:4])-1)+ '/12'
  
  else:
    reportBegin= 'ERROR'
    reportEnd= 'ERROR'
  return reportBegin, reportEnd

In [0]:
############
def findConst(report, linhaAtuacao):
  constatType="constatacao|informacao"
  #COLLECT the NUMBER of Constat
  # constat (NumC, TypeC, TitleC, FactC, NextNumC)
  constat=[]
  

  ###############
  # Report ARG
  if linhaAtuacao=='Avaliação dos Resultados da Gestão':
    fountParsePattern=0
    if bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')', report) ):
      fountParsePattern=1
      constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')', report)
      # constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+').*?fato', report)
        
      for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao|3\.\sconclusao)', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao|3\.\sconclusao)', report)[0] )
          elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(recomendacoes)', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(recomendacoes)', report)[0] )
          else:
            constat=[('ERROR', 'ERROR-LastConst1', 'ERROR', 'ERROR', 'ERROR')]
        else :
          if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report) ):
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report)[0] )
          elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report) ): # Reports without field FATO
            constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report)[0] )
          else:
            constat=[('ERROR', 'ERROR-beetwenConst', 'ERROR', 'ERROR', 'ERROR')]


    if bool( re.search('2\.\sresultados\sdos\strabalhos(.*?3\.\sconclusao)', report) ):
      fountParsePattern=1
      textBeetwen=re.findall('2\.\sresultados\sdos\strabalhos(.*?3\.\sconclusao)', report)

      constatText=textBeetwen[0]
      constatLST= re.findall('\s(\d\.\d+|\d\.\d+\.)\s', constatText)
      # constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)

      if len(constatLST)==0:
        constat.append(('2.', '', '', constatText, ''))
      else:
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(3\.\sconclusao)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(3\.\sconclusao)', constatText)[0]
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst2', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))

    elif bool( re.search('\sresultados\sdos\sexames(.*?\sconclusao)', report) ) or bool( re.search('\sresultados\sdos\sexames(.*?\srecomendacoes)', report) ) :
      fountParsePattern=1
      if bool( re.search('\sresultados\sdos\sexames(.*?\srecomendacoes)', report) ):
        textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\srecomendacoes)', report)
      else:
        if bool( re.search('\sresultados\sdos\sexames(.*?\sconclusao)', report) ):
          textBeetwen=re.findall('\sresultados\sdos\sexames(.*?\sconclusao)', report)

      if len(textBeetwen)>=2:
        constatSummary=textBeetwen[0]
        constatText=textBeetwen[1]
        constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatSummary)

        titles=[]
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendacoes|conclusao)', constatSummary) ):
              titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendacoes|conclusao)', constatSummary)[0] )
            else:
              titles.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary) ):
              titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary)[0] )
            else:
              titles.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))


        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes|conclusao)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes|conclusao)', constatText)[0]
              constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))

      elif len(textBeetwen)==1:
        constatText=textBeetwen[0]
        constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)
        for n in range(0, len(constatLST)):
        #LAST CONST goes until Conclusion
          if n == (len(constatLST)-1) :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes|conclusao)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes|conclusao)', constatText)[0]
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
          else :
            if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
              constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
              constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
            else:
              constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))



    if fountParsePattern==0: 
      constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]


  ##############
  #FOR 'RELATORIO DE AVALIACAO' 
  elif "relatorio de avaliacao" in report:
    textBeetwen=re.findall('resultados\sdos\sexames(.*?recomendacoes)', report)

    if len(textBeetwen)>=2:
      constatSummary=textBeetwen[0]
      constatText=textBeetwen[1]
      constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatSummary)

      titles=[]
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendacoes)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)(recomendacoes)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary) ):
            titles.append( re.findall('('+constatLST[n].replace('.','\.')+')\s(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatSummary)[0] )
          else:
            titles.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))


      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes)', constatText)[0]
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], titles[n][1], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))

    elif len(textBeetwen)==1:
      constatText=textBeetwen[0]
      constatLST= re.findall('\s(\d+\.|\d\.\d+)\s', constatText)
      for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
        if n == (len(constatLST)-1) :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)(recomendacoes)', constatText)[0]
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-LastConst', 'ERROR', 'ERROR', 'ERROR'))
        else :
          if bool( re.search('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText) ):
            constFind=re.findall('('+constatLST[n].replace('.','\.')+')\s()()(.*?)('+constatLST[n+1].replace('.','\.')+'\s)', constatText)[0] 
            constat.append( (constFind[0], constFind[1], constFind[2], constFind[3], constFind[4] ))
          else:
            constat.append(('ERROR', 'ERROR-Rel_Aval-beetwenConst', 'ERROR', 'ERROR', 'ERROR'))
    else: 
      constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]
  
  
  ###############
  # NORMAL:
  elif bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')', report) ):
    constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')', report)
    # constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+').*?fato', report)
      
    for n in range(0, len(constatLST)):
      #LAST CONST goes until Conclusion
      if n == (len(constatLST)-1) :
        if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report) ):
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report)[0] )
        else:
          constat=[('ERROR', 'ERROR-LastConst', 'ERROR', 'ERROR', 'ERROR')]
      else :
        if bool( re.search('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report) ):
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)('+constatLST[n+1][0]+')', report)[0] )
        elif bool( re.search('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report) ): # Reports without field FATO
          constat.append( re.findall('('+constatLST[n][0]+')\s('+constatType+')()(.*?)('+constatLST[n+1][0]+')', report)[0] )
        else:
          constat=[('ERROR', 'ERROR-beetwenConst', 'ERROR', 'ERROR', 'ERROR')]
  else:
    constat=[('ERROR', 'ERROR-NoConst', 'ERROR', 'ERROR', 'ERROR')]

  #CONCLUSION
  if bool( re.search('(iii|3\.)\s(conclusao)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4}\.)', report) ):    
    constat.append( re.findall('(iii|3\.)\s(conclusao)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4}\.)', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4})', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)(\d{4})', report)[0] )
    # constat.append( re.findall('(iii)\s(conclusao)()(.*?)\s(\d{4})', report)[0] )

  # constat (NumC, TypeC, TitleC, FactC, NextNumC)
  return constat

In [0]:
#
def findOrgSub(reportHEADER):
 
  beginText='(unidade\sauditada:|unidade\sauditada\s:)'
  endText='(codigo|municipio)'

  beginTextRA='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRA='(codigo|municipio)'
 
  beginTextRF='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRF='(montante\sde)'

  beginTextRAC='(unidade\sexaminada:|unidade\sexaminada\s:)'
  endTextRAC='(municipio)'
  #General Case
  if bool( re.search(beginText+'\s(.*?)'+endText, reportHEADER) ) :
      orgSub=re.findall(beginText+'\s(.*?)'+endText, reportHEADER)[0][1]

  elif (bool( re.search(beginTextRA+'\s(.*?)'+endTextRA, reportHEADER) )) & ("relatorio de avaliacao" in reportHEADER) :
    orgSub=re.findall(beginTextRA+'\s(.*?)'+endTextRA, reportHEADER)[0][1]
  
  elif (bool( re.search(beginTextRF+'\s(.*?)'+endTextRF, reportHEADER) )) & ("relatorio de fiscalizacao" in reportHEADER) :
    orgSub=re.findall(beginTextRF+'\s(.*?)'+endTextRF, reportHEADER)[0][1]
  
  elif (bool( re.search(beginTextRAC+'\s(.*?)'+endTextRAC, reportHEADER) )) & ("relatorio de acao de controle" in reportHEADER) :
    orgSub=re.findall(beginTextRAC+'\s(.*?)'+endTextRAC, reportHEADER)[0][1]

  else:
    orgSub='ERROR'

  return orgSub.strip()

#
def findOrgSubCod(orgSub, dfFile):
  orgSubCod=""
  for index, row in dfFile.iterrows():
    orgSubTest=(row['Nome Órgão Subordinado']).decode("utf-8")
    if orgSub.find(orgSubTest) != -1 or orgSubTest.find(orgSub) != -1:
      orgSub=orgSubTest
      orgSubCod=row['Código Órgão Subordinado']
  return orgSubCod

In [0]:
# # fileNumberLST=["11573","12564","13231","13349","13359","13319","13374","12375","8834" ]
# fileNumberLST=['13175','13465','13422','12963']
# fileNumberLST=['13555']
# DEBUG=1

dfFile['fileNumber']=dfFile['Links'].str.split('/').str[-1].str.split('.').str[0]
fileNumberLST=dfFile['fileNumber']
DEBUG=0

organizeLst=[]
i=0
for fileNumber in fileNumberLST:
  if ( len(dfSTR[dfSTR['FileName']==(fileNumber + ".pdf")])>0 ):
    report = dfSTR['Content'][ dfSTR['FileName']==(fileNumber + ".pdf")  ].reset_index(drop=True)[0]
    report = cleanReport(report)
    reportHeader = findHeader(report)
    reportNum = findReportNumber(reportHeader)
    reportBegin, reportEnd = findPeriod(reportHeader, reportNum)  
    orgSub = findOrgSub(reportHeader)
    orgSubCod = findOrgSubCod(orgSub, dfOrgSub)
    reportType = findReportType(fileNumber, dfFile)

    linhaAtuacao = dfFile[ dfFile['Links'].str.contains('/' + fileNumber + '.pdf') ]['Linha de Atuação'].reset_index(drop=True)[0]
    constat = findConst(report, linhaAtuacao)
       
    if ( i%100 == 0 ):
       print("\n" + str(i), end='' )
    i+=1
    print(".", end='')
  
    if DEBUG==1:
      lstVar=['report', 'reportHeader','reportNum','reportBegin','reportEnd',
              'orgSub','reportType']
      for var1 in lstVar:
        print(var1 + ': ' + str(len(eval(var1))) + ' - ' + eval(var1)[0:4])
      # print(constat[0])
  else:
    print('\nFile Parsed not found: ' + fileNumber )

  #Mount DF  
  for numC, typeC, titleC, textC, nextNumC in constat:
    if len(numC) > 0:
      # labelTypeC = 1 if typeC=='constatacao' else 0
      organizeLst.append([orgSubCod,orgSub,fileNumber,reportNum,reportType,reportBegin,reportEnd,numC,typeC, titleC, textC])
  columns=['OrgSubCod','OrgSub','FileNumber','ReportNumber','ReportType','DateBegin','DateEnd','ConstNumber','ConstTYPE','ConstTitle','ConstText']
  dfReportInColumns = pd.DataFrame(organizeLst, columns=columns)



0....................................................................................................
100....................................................................................................
200....................................................................................................
300....................................................................................................
400....................................................................................................
500....................................................................................................
600....................................................................................................
700....................................................................................................
800....................................................................................................
900..............................................................

In [0]:
# dfReportInColumns[ dfReportInColumns['ConstText']=='ERROR' ].head(10)
# dfReportInColumns[ dfReportInColumns['FileNumber']=='13014' ].head(10)
# len(dfReportInColumns[ dfReportInColumns['ConstText']!='ERROR' ])
dfReportInColumns

In [0]:
#VALIDATE RESULTS
print('Total Reports       : ' + str(len(fileNumberLST)))
print('Total Reports Parsed: ' + str(len(dfReportInColumns.FileNumber.unique())))
print('ERROR ReportNumber  : ' + str(len(dfReportInColumns[ (dfReportInColumns.ReportNumber=='ERROR') ].FileNumber.unique() )))
print('ERROR Date          : ' + str(len(dfReportInColumns[ (dfReportInColumns.DateBegin=='ERROR') ].FileNumber.unique() )))
print('ConstNumber OK      : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique() )))
print('ERROR ConstNumber   : ' + str(len(fileNumberLST) - len(dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') ].FileNumber.unique() )))
print('Number ConstatTexts : ' + str(len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ] )))

print('Objective: > 80 %')
print('Accuracy : ' + str( 100*(1-( (len(fileNumberLST)-len(dfReportInColumns[ (dfReportInColumns.ConstNumber!='ERROR') ].FileNumber.unique()))/len(fileNumberLST)))))


Total Reports       : 1058
Total Reports Parsed: 1051
ERROR ReportNumber  : 36
ERROR Date          : 29
ConstNumber OK      : 1009
ERROR ConstNumber   : 782
Number ConstatTexts : 13018
Objective: > 80 %
Accuracy : 95.36862003780719


In [0]:
#Write DF to disk
dfReportInColumns.to_pickle( FINAL_FILE_DF )

In [0]:
#### TEST
# print(len(fileNumberLST))
# print(len(dfReportInColumns.FileNumber.unique()))
# dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') & (dfReportInColumns.OrgSub!='ERROR') ] 
# dfReportInColumns[ dfReportInColumns.FileNumber=='13284' ] 
# dfReportInColumns[ dfReportInColumns.FileNumber=='13257' ] 
# len(dfReportInColumns[ (dfReportInColumns.ConstNumber=='ERROR') ] )


# print('Total Reports       : ' + str(len(fileNumberLST)))
# print('Total Reports Parsed: ' + str(len(dfReportInColumns.FileNumber.unique())))
# print('Total Reports ERRORS: ' + str(len(dfReportInColumns[ (dfReportInColumns.ReportNumber.str.contains('ERROR')) ].FileNumber.unique() )))
# print('Objective: > 80 %')
# print('Accuracy : ' + str( 100*(1-( len(dfReportInColumns[ (dfReportInColumns.ReportNumber.str.contains('ERROR')) ].FileNumber.unique())/len(fileNumberLST)))))

# dfReportInColumns2=dfReportInColumns
# dfReportInColumns[ dfReportInColumns.ReportNumber.str.contains('ERROR') ].head(15)
# dfReportInColumns

In [0]:
#TESTs
# fileNumber='12963'
fileNumber='13014'
# fileNumber='13175'
report = dfSTR['Content'][ dfSTR['FileName']==(fileNumber + ".pdf")  ].reset_index(drop=True)[0]
report = cleanReport(report)
constatType="constatacao|informacao"
constatLST=[]
if bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')', report) ):
  constatLST=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')', report)

constatLST
# for n in range(0, len(constatLST)):
#   x=re.findall('('+constatLST[n][0]+')\s('+constatType+')(.*?)fato(.*?)(recomendacoes)', report)[0]

# x


[('1.1.1.1', 'constatacao'),
 ('1.1.1.2', 'constatacao'),
 ('1.1.1.3', 'constatacao'),
 ('2.1.1.1', 'informacao'),
 ('2.1.1.2', 'constatacao'),
 ('2.1.1.3', 'constatacao'),
 ('1.1.1.1', 'constatacao'),
 ('1.1.1.2', 'constatacao'),
 ('1.1.1.3', 'constatacao'),
 ('2.1.1.1', 'informacao'),
 ('2.1.1.2', 'constatacao'),
 ('2.1.1.3', 'constatacao')]

In [0]:

# textBeetwen=re.findall('2\.\sresultados\sdos\strabalhos(.*?3\.\sconclusao)', report)
# # len(textBeetwen[0])
# constatText=textBeetwen[0]
# constatLST= re.findall('\s(\d\.\d+)\s', constatText)
# # constatLST
# # len(constatLST)
# # re.findall('(iii|3\.)\s(conclusao)()(.*?)(/[a-z]{2},\s\d|de\s\d{4}|/\d{4}\.)', report)[0] 
# # for n in range(0, len(constatLST)-1):
# constFind=re.findall('('+constatLST[15].replace('.','\.')+')\s()()(.*?)(3\.\sconclusao)', constatText)

# constatLST[15].replace('.','\.')
# constFind
# textBeetwen
# reportHeader = findHeader(report)
# reportHeader
# reportNum = findReportNumber(reportHeader)
# reportNum
# # # # reportBegin, reportEnd = findPeriod(reportHeader, reportNum) 
# orgSub = findOrgSub(reportHeader)
# orgSub
# print(reportBegin + ' ' + reportEnd)
# constat = findConst(report)
# # report
# if "relatorio de avaliacao" in report:
#     print("relatorio de avaliacao dos resultados da gestao")
# dfSTR
# fileName1='/' + fileNumber + '.pdf'
# fileName1='/13175.pdf'
# dfFile[ dfFile['Links'].str.contains('/' + fileNumber + '.pdf') ]['Linha de Atuação'][0]
# dfFile
# startText='(relatorio)'
# endText='(2\.\sresultados\sdos\sexames)'
# startTextRF='(relatorio)'
# # endTextRF='(2\.\sresultados\sdos\sexames|ii\s\resultado\sdos\sexames|ii\s\resultado\sdos\sexames)'
# endTextRF='(ii\sresultado\sdos\sexames)' 
# re.findall(startTextRF + '(.*?)'+ endTextRF,report)
# # report


In [0]:
###################################################
#OLD CODE
##################################################

# def findConst(report):
#   constatType="constatacao|informacao"
#   if bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')(.*?)fato(.*?)(\d\.\d\.\d\.\d+\s)', report) ) :
#     constat=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')(.*?)fato(.*?)(\d\.\d\.\d\.\d+\s)', report)
  
#   #JUST ONE CONSTATION and the CONCLUSION
#   elif bool( re.search('(\d\.\d\.\d\.\d+)\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report) ) :
#     constat=re.findall('(\d\.\d\.\d\.\d+)\s('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report)
    
#   else:
#     constat=[('ERROR', 'ERROR', 'ERROR', 'ERROR', 'ERROR')]

  
#   #LAST CONSTATATION
#   lastConst=constat[ len(constat)-1 ][4]
#   if bool( re.search('('+lastConst+')('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report) ): 
#     constat.append( re.findall('('+lastConst+')('+constatType+')(.*?)fato(.*?)(iii\sconclusao)', report)[0] )

#   #CONCLUSION
#   if bool( re.search('(iii)\s(conclusao)()(.*?)\s(\d{4})', report) ):
#     constat.append( re.findall('(iii)\s(conclusao)()(.*?)\s(\d{4})', report)[0] )

#   # constat (NumC, TypeC, TitleC, FactC, NextNumC)
#   return constat

In [0]:
# def findOrgSub(reportHEADER):

#   # # Unidade Examinada: (Relatorio de Avaliação)
#   # if "relatorio de avaliacao" in reportHEADER:
#   #   beginText='(unidade\sexaminada:|unidade\sexaminada\s:)'
#   #   if if bool( re.search(beginText+'\s(.*?)'+endText, reportHEADER) ) :
#   #     orgSub=re.findall(beginText+'\s(.*?)'+endText, reportHEADER)[0][1]
#   #   else:
#   #     orgSub='ERROR'
#   # else:    
#   #   if bool( re.search(beginText+'\s(.*?)'+endText, reportHEADER) ) :
#   #     orgSub=re.findall(beginText+'\s(.*?)'+endText, reportHEADER)[0][1]
#   #   else:
#   #     orgSub='ERROR'
  
  
#   return orgSub.strip()